In [1]:
import pandas as pd
import numpy as np

In [2]:
mush_data = pd.read_csv('mushroom_data_all.csv')
mush_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   class_edible              8124 non-null   object
 1   cap-shape                 8124 non-null   object
 2   cap-surface               8124 non-null   object
 3   cap-color                 8124 non-null   object
 4   bruises                   8124 non-null   object
 5   odor                      8124 non-null   object
 6   gill-attachment           8124 non-null   object
 7   gill-spacing              8124 non-null   object
 8   gill-size                 8124 non-null   object
 9   gill-color                8124 non-null   object
 10  stalk-shape               8124 non-null   object
 11  stalk-root                8124 non-null   object
 12  stalk-surface-above-ring  8124 non-null   object
 13  stalk-surface-below-ring  8124 non-null   object
 14  stalk-color-above-ring  

In [3]:
mush_data = mush_data.replace(['p', 'e'], ['0', '1'])

Convertito i valori poisonous ed edible in 0 ed 1 rispettivamente

Nel dataset non sono presenti valori nulli per nessuna colonna, ma per la colonna 11 (stalk-root) ci sono dei possibili missing value codificati con il valore '?'

Utilizzando get_dummies è più difficile fare il plot

In [4]:
mush_data['stalk-root'].value_counts()

b    3776
?    2480
1    1120
c     556
r     192
Name: stalk-root, dtype: int64

In [5]:
mush_data.describe()

,class_edible,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
count,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,...,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124
unique,2,6,4,10,2,9,2,2,2,12,...,4,9,9,1,4,3,5,9,6,7
top,1,x,y,n,f,n,f,c,b,b,...,s,w,w,0,w,o,0,w,v,d
freq,4208,3656,3244,2284,4748,3528,7914,6812,5612,1728,...,4936,4464,4384,8124,7924,7488,3968,2388,4040,3148


I valori mostrati dal describe sono parziali poiché non è possibile ad esempio calcolare il min/max su stringhe

In [6]:
temp_data = pd.DataFrame()
for column in mush_data.drop('class_edible', axis=1).columns:
    temp_data = pd.concat([temp_data, pd.get_dummies(mush_data[column], prefix=column)], axis=1)
    mush_data = mush_data.drop(column, axis=1)
mush_data = pd.concat([temp_data, mush_data], axis=1)

Da correggere, metodo subottimale per codificare le feature categoriche in interi 
vedere: https://towardsdatascience.com/encoding-categorical-features-21a2651a065c

In [7]:
mush_data.shape

(8124, 118)

In [8]:
mush_data

,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,cap-surface_s,cap-surface_y,...,population_v,population_y,habitat_0,habitat_d,habitat_g,habitat_l,habitat_m,habitat_u,habitat_w,class_edible
0,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
2,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1
3,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1
8120,0,0,0,0,0,1,0,0,1,0,...,1,0,0,0,0,1,0,0,0,1
8121,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1
8122,0,0,0,1,0,0,0,0,0,1,...,1,0,0,0,0,1,0,0,0,0


In [9]:
from sklearn.model_selection import train_test_split

X = mush_data.drop(['class_edible'], axis=1)
y = mush_data['class_edible']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, train_size=0.7, test_size=0.3)

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import accuracy_score


model = DecisionTreeClassifier(max_depth=7, random_state=1)
model.fit(X_train, y_train)

#predictions = model.predict(X_test)
#print('Accuracy Score: ', accuracy_score(y_test, predictions) * 100, '%')
print('Training Accuracy: ', model.score(X_train, y_train) * 100, '%')
print('Testing Accuracy: ', model.score(X_test, y_test) * 100, '%')

Training Accuracy:  100.0 %
Testing Accuracy:  100.0 %
